In [77]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [78]:
df=pd.read_csv('Titanic-Dataset.csv')

In [79]:
df.head()

PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]

In [80]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [81]:
df.head()

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S

In [82]:
x=df.drop(columns=['Survived'])
y=df['Survived']

In [83]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=2)

In [84]:
x_train.head()

Pclass     Sex   Age  SibSp  Parch      Fare Embarked
451       3    male   NaN      1      0   19.9667        S
345       2  female  24.0      0      0   13.0000        S
687       3    male  19.0      0      0   10.1708        S
279       3  female  35.0      1      1   20.2500        S
742       1  female  21.0      2      2  262.3750        C

In [85]:
numerical_features=['Age','Fare']
numerical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaling',StandardScaler())
])

categorical_features=['Sex','Embarked']
categorical_transfomer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('OneHotEncoder',OneHotEncoder(handle_unknown='ignore'))
])

In [86]:
preprocessor=ColumnTransformer(transformers=[
    ('num',numerical_transformer,numerical_features),
    ('cat',categorical_transfomer,categorical_features)
])

In [87]:
clf=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',LogisticRegression())
])

In [88]:
from sklearn import set_config
set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked'])])),
                ('classifier', LogisticRegression())])

In [89]:
param_grid={
    'preprocessor__num__imputer__strategy': ['mean','median'],
    'preprocessor__cat__imputer__strategy': ['most_frequent','constant'],
    'classifier__C':[0.1,1,10,100]
}
grid_search=GridSearchCV(clf,param_grid,cv=10)

In [90]:
grid_search.fit(x_train,y_train)
print(f"best params:")
print(grid_search.best_params_)

best params:
{'classifier__C': 1, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean'}
